##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Normalizaciones

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/layers_normalizations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>


## Descripción general

En estas anotaciones se proporciona una breve introducción a las [capas de normalización](https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py) de TensorFlow. Las capas compatibles actualmente son las siguientes:

- **Normalización por grupos** (complementos de TensorFlow)
- **Normalización por instancias** (complementos de TensorFlow)
- **Normalización por capas** (esencial de TensorFlow)

La idea de base detrás de estas capas es la de normalizar la salida de una capa de activación para mejorar la convergencia durante el entrenamiento. En contraste con la [normalización por lotes](https://keras.io/layers/normalization/), estas normalizaciones no funcionan en lotes, sino que normalizan las activaciones de a una sola muestra; por lo tanto, resultan adecuadas también para redes neuronales recurrentes.

Por lo general, la normalización se lleva a cabo mediante el cálculo de la media y la desviación estándar de un subgrupo en el tensor de entrada. También es posible aplicar un factor escalar y uno de compensación en este caso.

$y_{i} = \frac{\gamma ( x_{i} - \mu )}{\sigma }+ \beta$

$ y$ : salida

$x$ : entrada

$\gamma$ : factor escalar

$\mu$: media

$\sigma$: desviación estándar

$\beta$: factor de desplazamiento (<em>offset</em>)

Con la siguiente imagen se demuestra la diferencia entre estas técnicas. Cada subgráfico muestra un tensor de entrada donde N es el eje del lote; C, como eje del canal; y H (alto) y W (ancho), como los ejes espaciales (por ejemplo, el alto y el ancho de una foto). Los pixeles en azul están normalizados por la misma media y varianza, calculada por el agregado de los valores de estos pixeles.

![](https://github.com/shaohua0116/Group-Normalization-Tensorflow/raw/master/figure/gn.png)

Fuente: (https://arxiv.org/pdf/1803.08494.pdf)

Los pesos gamma y beta se pueden entrenar en cualquiera de las capas de normalización para compensar por la posible pérdida de la capacidad de representación. Estos factores se pueden activar estableciendo el código `center` o la marca `scale` como `True`. Por supuesto, se pueden usar `initializers`, `constraints` y `regularizer` para `beta` y `gamma` a fin de ajustar estos valores durante el proceso de entrenamiento. 

## Preparación

### Instalación de los componentes de Tensorflow 2.0 y Tensorflow

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

### Preparación del conjunto de datos

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Tutorial para la normalización por grupos

### Introducción

En la normalización por grupos (GN), los canales de las entradas se dividen en subgrupos más pequeños y se normalizan estos valores, según su media y varianza. Dado que la normalización por grupos funciona con ejemplos únicos, podemos decir que esta técnica es independiente del tamaño del lote.

La normalización por grupos a nivel experimental obtuvo una calificación cercana a la de la normalización por lotes en cuanto a tareas de clasificación de imágenes. Por lo tanto, puede ser beneficioso usar la normalización por grupos, en vez de la normalización por lotes, en aquellos casos en que el tamaño del lote en general es pequeño (<em>low</em>), algo que podría derivar en un mal funcionamiento de la normalización por lotes.

### Ejemplo en el que se dividen 10 canales de una capa Conv2D en 5 subgrupos en un escenario estándar "<em>channels last</em>" (los canales al final):

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # Groupnorm Layer
  tfa.layers.GroupNormalization(groups=5, axis=3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Tutorial de normalización por instancias

### Introducción

La normalización por instancias es un caso especial de normalización por grupos en el que el tamaño del grupo es igual al tamaño del canal (o del tamaño del eje).

Los resultados experimentales muestran que la normalización por instancias funciona bien en la transferencia de estilos cuando se usa en lugar de la normalización por lotes. Recientemente, la normalización por instancias también se ha usado como reemplazo de la normalización por lotes en redes GAN.

### Ejemplo

Aplicación de InstanceNormalization después de una capa Conv2D y uso de un factor escalar y de compensación inicializado uniforme.

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tfa.layers.InstanceNormalization(axis=3, 
                                   center=True, 
                                   scale=True,
                                   beta_initializer="random_uniform",
                                   gamma_initializer="random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Tutorial de normalización por capas

### Introducción

La normalización por capas es un caso especial de normalización por grupos en la que el tamaño del grupo es 1. La media y la desviación estándar se calcula a partir de todas las activaciones de una muestra individual.

Los resultados experimentales muestran que la normalización por capas es adecuada para las redes neuronales recurrentes, ya que funcionan con independencia del tamaño del lote.

### Ejemplo

Aplicación de la normalización por capas después de una capa Conv2D y uso de un factor escalar y de compensación. 

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tf.keras.layers.LayerNormalization(axis=3 , center=True , scale=True),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Bibliografía

[Layer norm](https://arxiv.org/pdf/1607.06450.pdf) (Normalización por capas)

[Instance norm](https://arxiv.org/pdf/1607.08022.pdf) (Normalización por instancias)

[Group Norm](https://arxiv.org/pdf/1803.08494.pdf) (Normalización por grupos)